In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from torchvision import transforms
import os
import torch
from PIL import Image

test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
                                     ])
folder_path = '/content/drive/MyDrive/Colab Notebooks/test'

In [3]:
results = []
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")
import time
model = torch.load('/content/drive/MyDrive/Colab Notebooks/checkpoint/full-image-last-1.000.pth').to(device)
for filename in os.listdir(folder_path):
    if filename.endswith(".png"):
        img_path = os.path.join(folder_path, filename)
        img = Image.open(img_path)
        img = img.convert('RGB')
        x = test_transform(img)
        x = x.unsqueeze(0).to(device)
        #  predict
        st = time.time()
        with torch.no_grad():
            predictions = model(x)
        _, predicted = torch.max(predictions.data, 1)
        results.append((filename, predicted.item()))
        et = time.time()
et - st

Using device: cuda:0


0.008114099502563477

In [4]:
import re

results = [(x[0], x[1] + 1) for x in results]
sorted_list = sorted(results, key=lambda x: int(re.search(r'\d+', x[0]).group()))

sorted_list_desc = sorted(results, key=lambda x: int(re.search(r'\d+', x[0]).group()))

In [5]:
sorted_list_desc

[('1.png', 1),
 ('2.png', 2),
 ('3.png', 1),
 ('4.png', 5),
 ('5.png', 5),
 ('6.png', 2),
 ('7.png', 3),
 ('8.png', 1),
 ('9.png', 3),
 ('10.png', 3),
 ('11.png', 5),
 ('12.png', 5),
 ('13.png', 1),
 ('14.png', 4),
 ('15.png', 2),
 ('16.png', 2),
 ('17.png', 2),
 ('18.png', 4),
 ('19.png', 5),
 ('20.png', 4),
 ('21.png', 5),
 ('22.png', 2),
 ('23.png', 4),
 ('24.png', 1),
 ('25.png', 5),
 ('26.png', 4),
 ('27.png', 5),
 ('28.png', 2),
 ('29.png', 2),
 ('30.png', 4),
 ('31.png', 3),
 ('32.png', 3),
 ('33.png', 3),
 ('34.png', 3),
 ('35.png', 1),
 ('36.png', 3),
 ('37.png', 3),
 ('38.png', 5),
 ('39.png', 1),
 ('40.png', 1),
 ('41.png', 5),
 ('42.png', 3),
 ('43.png', 4),
 ('44.png', 2),
 ('45.png', 5),
 ('46.png', 3),
 ('47.png', 1),
 ('48.png', 4),
 ('49.png', 2),
 ('50.png', 3)]

In [10]:
import json

def submit_to_json(resultlist, stuId):
    predictresult = []
    for image_name, label in resultlist:
        predictresult.append({
            "title": image_name,
            "answer": str(label),
            "score": float(0.0)
        })

    submit_data = {"stuId": stuId, "answers": predictresult}
    with open("/content/drive/MyDrive/Colab Notebooks/pred_results.json", "w", encoding='utf-8') as file:
        json.dump(submit_data, file, ensure_ascii=False)

submit_to_json(resultlist=sorted_list_desc, stuId=3122009322)


with open('/content/drive/MyDrive/Colab Notebooks/label.json') as f1:
    data1 = json.load(f1)

with open('/content/drive/MyDrive/Colab Notebooks/pred_results.json') as f2:
    data2 = json.load(f2)

answers1 = [answer['answer'] for answer in data1['answers']]
answers2 = [answer['answer'] for answer in data2['answers']]

agreement_rate = sum(a1 == a2 for a1, a2 in zip(answers1, answers2)) / len(answers1)
print(f'The agreement rate is {agreement_rate:.2f}')

The agreement rate is 0.84
